In [5]:
import pandas as pd

data = pd.read_csv('../data/enriquecidos-004.pkl')
data.drop('Unnamed: 0', axis=1, inplace=True)
data.head()

,ID,DATE,DELTA,READING,PRECIPITATIONS,MIN_TEMP,MEAN_TEMP,MAX_TEMP,SUN,WEEK_DAY,IS_WEEKEND,sin_WEEK_DAY,cos_WEEK_DAY,IS_PUBLIC_HOLIDAY
0,0,2019-02-01,243.0,331953.0,0.0,16.5,17.6,18.7,7.1,4,0,-0.433884,-0.900969,0
1,0,2019-02-02,236.0,332189.0,0.0,9.8,13.4,17.1,6.1,5,1,-0.974928,-0.222521,0
2,0,2019-02-03,335.0,332524.0,0.0,7.7,10.6,13.6,9.3,6,1,-0.781831,0.623490,0
3,0,2019-02-04,252.0,332776.0,0.0,4.1,10.6,17.2,9.3,0,0,0.000000,1.000000,0
4,0,2019-02-05,220.0,332996.0,0.0,7.5,14.6,21.6,9.2,1,0,0.781831,0.623490,0


In [6]:
def num_days(df, i):
    data_one_id = df[df['ID']==i]
    return len(data_one_id['DATE'].unique())

In [8]:
contadores_completos = []
for i in data['ID'].unique():
    if num_days(data, i) == 365:
        contadores_completos.append(i)

In [9]:
len(contadores_completos)

1975

In [11]:
data = data[data['ID'].isin(contadores_completos)]

In [23]:
def num_decreasing_reading(data, contador):
    rdif0 = data[data['ID']==contador]['READING'].diff().fillna(0).round(4)
    errores = 0
    for (i, rdif) in enumerate(rdif0):
        if rdif < 0:
            errores += 1
    return errores

def num_readingdiff_not_eq_delta(df, contador):
    rdif0 = df[df['ID']==contador]['READING'].diff().fillna(0).round(4)
    delta0 = df[df['ID']==contador]['DELTA']
    errores = 0
    for (i, (rdif, delta)) in enumerate(zip(rdif0, delta0)):
        if rdif!=delta:
            errores += 1
    return errores

### Contadores con READING decreasing
Como son pocos y en muy poca proporción voy a ignorar este error

In [30]:
from tqdm import tqdm

bondad_contadores = {}
for contador in tqdm(data['ID'].unique()):
    rdif0 = data[data['ID']==contador]['READING'].diff().fillna(0).round(4)
    errores = 0
    for (i, rdif) in enumerate(rdif0):
        if rdif < 0:
            errores += 1
    bondad_contadores[contador] = errores/len(rdif0)

100%|█████████████████████████████████████████████████████████████████████████████| 1975/1975 [00:02<00:00, 910.32it/s]


In [36]:
sum([1 if i!=0 else 0 for i in bondad_contadores.values()])

58

### Meter nueva variable consumo total del contador al cabo del año

In [63]:
def consumo_total(df, i):
    df0=df[df['ID']==i]
    initial_reading = df0[df0['DATE']=="2019-02-01"].READING.values[0]
    final_reading = df0[df0['DATE']=="2020-01-31"].READING.values[0]
    return final_reading-initial_reading

In [64]:
def consumo_total2(i):
    return consumo_total(data, i)

In [67]:
from tqdm import tqdm
tqdm.pandas()

data['TOTAL_CONSUMPTION'] = data['ID'].progress_apply(consumo_total2)

100%|█████████████████████████████████████████████████████████████████████████| 720875/720875 [18:07<00:00, 662.57it/s]


### Dividir en train y test
Tomo para test las dos últimas semanas de Enero

In [71]:
import datetime
from tqdm import tqdm

'''
given a start date in datetime format "start_date" and an "end_date" returns a list of strings with the dates from
"start_date" to "end_date".

Example:

start_date = datetime.date(2019, 9 , 30)
end_date = datetime.date(2019, 10, 7)
get_date_range(start_date, end_date)
'''
def get_date_range(start_date, end_date):
    number_of_days = (end_date-start_date).days
    return [(start_date + datetime.timedelta(days = day)).isoformat() for day in range(number_of_days+1)]

In [79]:
start_date = datetime.date(2020, 1 , 18)
end_date = datetime.date(2020, 1, 31)
test = data[data['DATE'].isin(get_date_range(start_date, end_date))]

start_date = datetime.date(2019, 2 , 1)
end_date = datetime.date(2020, 1, 17)
train = data[data['DATE'].isin(get_date_range(start_date, end_date))]

In [104]:
import xgboost as xgb

model = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    max_depth=3
)

In [90]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()

In [ ]:
X_train = train.drop(['READING','DELTA','DATE','WEEK_DAY'], axis=1)
y_train = train['DELTA']

In [139]:
train_0 = train[train['ID']==0]
test_0 = test[test['ID']==0]

X_train = train_0.drop(['READING','DELTA','DATE','WEEK_DAY'], axis=1)
y_train = train_0['DELTA']
X_test = test_0.drop(['READING','DELTA','DATE','WEEK_DAY'], axis=1)
y_test = test_0['DELTA']

In [140]:
model.fit(X_train, y_train)

C:\Users\sergi\Anaconda3\envs\cajamar\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=3, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [ ]:
X_test = test.drop(['READING','DELTA','DATE','WEEK_DAY'], axis=1)
y_test = test['DELTA']

In [142]:
from sklearn.metrics import mean_squared_error

y_pred = model.predict(X_test)
y_real = y_test

print('MSE:', mean_squared_error(y_real, y_pred))

MSE: 8422.35959496506


In [112]:
dict_feature_importance = {'cols':X_train.columns,
                           'importances': model.feature_importances_}
pd.DataFrame.from_dict(dict_feature_importance).sort_values('importances', ascending=False)

,cols,importances
10,TOTAL_CONSUMPTION,0.320967
6,IS_WEEKEND,0.129901
0,ID,0.117354
3,MEAN_TEMP,0.104383
2,MIN_TEMP,0.096233
7,sin_WEEK_DAY,0.054312
8,cos_WEEK_DAY,0.049067
5,SUN,0.048700
4,MAX_TEMP,0.041115
1,PRECIPITATIONS,0.037556


In [133]:
y_real[0:7].values

array([421. , 271.2, 306. , 292. , 460. , 331. , 368. ])

In [135]:
y_pred[0:7]

array([266.96335, 268.00647, 275.99628, 268.89127, 269.38354, 268.2073 ,
       259.86816], dtype=float32)

In [129]:
print('MSE:', mean_squared_error(y_real[0:7], y_pred[0:7]))

MSE: 11020.253570281815


In [126]:
test.head(30)

,ID,DATE,DELTA,READING,PRECIPITATIONS,MIN_TEMP,MEAN_TEMP,MAX_TEMP,SUN,WEEK_DAY,IS_WEEKEND,sin_WEEK_DAY,cos_WEEK_DAY,IS_PUBLIC_HOLIDAY,TOTAL_CONSUMPTION
351,0,2020-01-18,421.00,430714.00,2.4,7.7,13.2,18.8,4.0,5,1,-0.974928,-0.222521,0,103013.0
352,0,2020-01-19,271.20,430987.00,38.2,9.5,9.6,9.6,0.0,6,1,-0.781831,0.623490,0,103013.0
353,0,2020-01-20,306.00,431293.00,53.5,5.0,7.1,9.2,0.0,0,0,0.000000,1.000000,0,103013.0
354,0,2020-01-21,292.00,431585.00,35.3,6.6,9.8,13.1,0.0,1,0,0.781831,0.623490,0,103013.0
355,0,2020-01-22,460.00,432045.00,9.9,9.7,12.0,14.3,0.0,2,0,0.974928,-0.222521,0,103013.0
356,0,2020-01-23,331.00,432376.00,9.9,5.4,9.9,14.4,3.7,3,0,0.433884,-0.900969,0,103013.0
357,0,2020-01-24,368.00,432744.00,0.0,8.0,12.0,16.0,0.6,4,0,-0.433884,-0.900969,0,103013.0
358,0,2020-01-25,390.00,433134.00,0.0,10.3,12.2,14.2,0.6,5,1,-0.974928,-0.222521,0,103013.0
359,0,2020-01-26,304.00,433438.00,0.0,9.8,13.8,17.8,6.2,6,1,-0.781831,0.623490,0,103013.0
360,0,2020-01-27,213.00,433651.00,0.0,7.6,13.0,18.4,7.6,0,0,0.000000,1.000000,0,103013.0
